In [1]:
*Cruise Ship Cruise Employees*

In [2]:
"""Linear Regression Model to Predict Number of Employees Required to man a newly manufactured cruise ship, based on ecommerce data"""

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('cruise_regression').getOrCreate()

In [5]:
#Load data
df = sqlContext.sql("SELECT * FROM cruise")

In [6]:
#Getting familiar with the data
df.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [7]:
for item in df.head(1)[0]:
  print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55

In [8]:
for ship in df.head(5):
  print(ship)
  print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)

In [9]:
%sql
SELECT * FROM cruise LIMIT 10

Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
Journey,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55
Quest,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55
Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.8,6.7
Conquest,Carnival,11,110.0,29.74,9.53,14.88,36.99,19.1
Destiny,Carnival,17,101.353,26.42,8.92,13.21,38.36,10.0
Ecstasy,Carnival,22,70.367,20.52,8.55,10.2,34.29,9.2
Elation,Carnival,15,70.367,20.52,8.55,10.2,34.29,9.2
Fantasy,Carnival,23,70.367,20.56,8.55,10.22,34.23,9.2
Fascination,Carnival,19,70.367,20.52,8.55,10.2,34.29,9.2
Freedom,Carnival,6,110.23899999999999,37.0,9.51,14.87,29.79,11.5


In [10]:
[df.where(df[column].isNull()).count() for column in df.columns]

Out[ 50 ]: [0, 0, 0, 0, 0, 0, 0, 0, 0]

In [11]:
df.groupBy(df['Cruise_line']).count().orderBy('count').show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Orient| 1|
 Disney| 2|
 Crystal| 2|
 Azamara| 2|
 Oceania| 3|
 Seabourn| 3|
 Cunard| 3|
 Windstar| 3|
 Silversea| 4|
Regent_Seven_Seas| 5|
 P&O| 6|
 Star| 6|
 MSC| 8|
 Celebrity| 10|
 Costa| 11|
 Norwegian| 13|
 Holland_American| 14|
 Princess| 17|
 Carnival| 22|
 Royal_Caribbean| 23|
+-----------------+-----+

In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
#ML Preprocessing step 1: String index the avatar categories (can also explore one hot encoding)
indexer = StringIndexer(inputCol = 'Cruise_line', outputCol = 'cruise_category')
indexed = indexer.fit(df).transform(df)
indexed.head(1)

Out[ 58 ]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_category=16.0)]

In [14]:
indexed.columns

Out[ 59 ]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_category']

In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
#ML Preprocessing step 2: Assemble features into single column
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_category'], outputCol = 'features')

output = assembler.transform(indexed)

In [17]:
output.columns

Out[ 62 ]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_category',
 'features']

In [18]:
#ML Preprocessing step 3: Select features & response variable
final_data = output.select(['features','crew'])
output.select(['features', 'crew']).show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [19]:
from pyspark.sql.functions import corr

In [20]:
#ML Preprocessing step 4: Check for high correlations between response variable & predictors for prediction power analysis
[df.select(corr('crew',column)).show() for column in df.columns] #passengers, cabins are highly correlated

+---------------------+
corr(crew, Ship_name)|
+---------------------+
 NaN|
+---------------------+

+-----------------------+
corr(crew, Cruise_line)|
+-----------------------+
 null|
+-----------------------+

+-------------------+
 corr(crew, Age)|
+-------------------+
-0.5306565039638852|
+-------------------+

+-------------------+
corr(crew, Tonnage)|
+-------------------+
 0.927568811544939|
+-------------------+

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

+------------------+
corr(crew, length)|
+------------------+
0.8958566271016579|
+------------------+

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+

+-----------------------------+
corr(crew, passenger_density)|
+-----------------------------+
 -0.15550928421699717|
+-----------------------------+

+----------------+
corr(crew, crew)|
+----------------+
 1.0|
+----------------+

 Out[ 101 ]: [None, None, None, None, None, None, None, None, None]

In [21]:
#Partition data
train_data, test_data = final_data.randomSplit([0.7,0.3], seed = 1234)
train_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 116|
 mean| 7.582241379310355|
 stddev|3.4363367973334116|
 min| 0.59|
 max| 19.1|
+-------+------------------+

In [22]:
from pyspark.ml.regression import LinearRegression

In [23]:
#Initiate linear regression model
model = LinearRegression(labelCol= 'crew')

In [24]:
#Train model
trained_model = model.fit(train_data)

In [25]:
#Evaluate model
results = trained_model.evaluate(test_data)

In [26]:
#Model performance
results.rootMeanSquaredError

Out[ 89 ]: 0.9093783989128907

In [27]:
results.r2

Out[ 90 ]: 0.936793966098222

In [28]:
results.residuals.show()

+--------------------+
 residuals|
+--------------------+
 0.6264975362281682|
 -1.3062943960637625|
 0.28718106321792547|
-0.35912405365133004|
 -0.5419336150775962|
 1.1798311453018702|
-0.31908273385299246|
-0.37326798734876476|
 -0.4362140907915677|
 2.049403630542688|
 -0.2921155878124537|
 0.990024797000471|
 0.9277189763763829|
 0.06268869080267125|
 0.9452375147060579|
-0.12709143165114778|
 0.08691083604959315|
 3.2785629671140004|
 -0.7570043682481806|
 -0.6651053382957883|
+--------------------+
only showing top 20 rows

In [29]:
results.meanSquaredError

Out[ 92 ]: 0.8269690724093726

In [30]:
results.meanAbsoluteError

Out[ 93 ]: 0.6516635531122323